In [12]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import utils
import config

# For GCN-LSTM model
from tensorflow import keras
from keras import Model, initializers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping
from tqdm.keras import TqdmCallback
import stellargraph as sg
from stellargraph.layer import GCN_LSTM

# For hyperopt (parameter optimization)
from hyperopt import Trials, STATUS_OK, tpe, fmin, hp
from hyperopt.pyll.base import scope  # quniform returns float, some parameters require int; use this to force int

# Evaluation metrics
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

%matplotlib inline

In [17]:
def format_data(data, weeks=1):
    X_total = data.values[:209, :]

    data = X_total[(53-weeks):, :]
    print(data.shape)
    scaler, values = utils.scale(data)
    X, y = utils.sequence_data_preparation(1, weeks, values.T)
    
    trainX = X[:140, :, :]
    trainY = y[:140, :]
    validX = X[140:148, :, :]
    validY = y[140:148, :]
    testX = X[148:, :, :]
    testY = y[148:, :]
    print(trainX.shape, testX.shape)

    return trainX, trainY, validX, validY, testX, testY

In [18]:
def format_scale_data(data, weeks=1):
    X_total = data.values[:209, :]

    data = X_total[(53-weeks):, :]
    scaler, values = utils.scale(data)
    X, y = utils.sequence_data_preparation(1, weeks, values.T)
    
    trainX = X[:140, :, :]
    trainY = y[:140, :]
    validX = X[140:148, :, :]
    validY = y[140:148, :]
    testX = X[148:, :, :]
    testY = y[148:, :]
    print(trainX.shape, testX.shape)
    y_scaler, y = utils.scale(data[:, 1].reshape((len(data), 1)))

    return trainX, trainY, validX, validY, testX, testY, scaler, y_scaler

In [4]:
def run():
    def f_nn(params):
        # Generate data with given window
        saltlake_week = pd.read_csv('../saltlake_week.csv')
        data = saltlake_week[['Cases', 'VMT (Veh-Miles)', 'News Sentiment', 'Unemployment Rate', 'PRCP', 'SNWD',
                              'Percent_Fully_Vaccinated_5&Older', 'TAVG',
                              'Stay at Home', 'Mask', 'School Opening', 'Health Emergency', 'Holiday']]
        trainX, trainY, validX, validY, testX, testY = format_data(data=data, weeks=params['weeks'])
        
        gc_layers = [params['gcn_units'] for i in range(params['gcn_layers'])]
        gc_act = [params['gcn_activation'] for i in range(params['gcn_layers'])]
        lstm_layers = [params['lstm_units'] for i in range(params['lstm_layers'])]
        lstm_act = [params['lstm_activation'] for i in range(params['lstm_layers'])]
        
        # Keras LSTM model
        model = GCN_LSTM(
                seq_len=config.SEQ_LEN,
                adj=config.TOTAL_ADJACENT_MATRIX,
                gc_layer_sizes=gc_layers,
                gc_activations=gc_act,
                lstm_layer_sizes=lstm_layers,
                lstm_activations=lstm_act,
                kernel_initializer=initializers.Identity(gain=1)
        )

        x_input, x_output = model.in_out_tensors()
        model = Model(inputs=x_input, outputs=x_output)
        lr = 0.001
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=params['learning_rate']), loss="mse",
              metrics=["mse"])

        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)

        result = model.fit(trainX, trainY, verbose=0, validation_data=(validX, validY),
                            batch_size=4,
                            epochs=200,
                            callbacks=[es, TqdmCallback(verbose=1)]
                            )

        # get the lowest validation loss of the training epochs
        validation_loss = np.amin(result.history['val_loss'])
        print('Best validation loss of epoch:', validation_loss)


        return {'loss': validation_loss, 'status': STATUS_OK, 'model': model, 'params': params}

    # hyperparameters to search over with hyperopt
    space = {'dropout': hp.uniform('dropout', 0.01, 0.5),
             'gcn_units': scope.int(hp.quniform('gcn_units', 4, 32, 4)),
             'gcn_layers': scope.int(hp.quniform('gcn_layers', 1, 2, 1)),
             'lstm_units': scope.int(hp.quniform('lstm_units', 8, 128, 4)),
             'lstm_layers': scope.int(hp.quniform('lstm_layers', 1, 6, 1)),
             'weeks': scope.int(hp.quniform('weeks', 1, 10, 1)), 
             'gcn_activation': hp.choice('gcn_activation', ['relu', 'sigmoid', 'tanh']), 
             'lstm_activation': hp.choice('lstm_activation', ['relu', 'sigmoid', 'tanh']),
             'learning_rate': hp.uniform('learning_rate', 0.001, 0.1)
             }

    trials = Trials()
    best = fmin(f_nn, space, algo=tpe.suggest, max_evals=50, trials=trials)

    # get best model
    best_model = trials.results[np.argmin([r['loss'] for r in trials.results])]['model']
    best_params = trials.results[np.argmin([r['loss'] for r in trials.results])]['params']

    print(best_params)
    print(best_model.summary())
    best_model.save('Model/GCN-LSTM')
    
    saltlake_week = pd.read_csv('../saltlake_week.csv')
    data = saltlake_week[['Cases', 'VMT (Veh-Miles)', 'News Sentiment', 'Unemployment Rate', 'PRCP', 'SNWD',
                          'Percent_Fully_Vaccinated_5&Older', 'TAVG', 
                          'Stay at Home', 'Mask', 'School Opening', 'Health Emergency', 'Holiday']]
    trainX, trainY, validX, validY, testX, testY, scaler, y_scaler = format_scale_data(data=data, weeks=best_params['weeks'])
    
    best_model.evaluate(testX, testY)
    
    yhat_test = best_model.predict(testX)
    yhat_test_inv = y_scaler.inverse_transform(yhat_test).reshape((-1, 1))
    testY_inv = y_scaler.inverse_transform(testY.reshape((-1, 1)))
    
    yhat_train = best_model.predict(trainX).reshape((-1, 1))
    yhat_train_inv = y_scaler.inverse_transform(yhat_train)
    trainY_inv = y_scaler.inverse_transform(trainY.reshape((-1, 1)))
    
    print("Mean Squared Error: {}".format(mean_squared_error(testY_inv, yhat_test_inv)))
    print("Root Mean Squared Error: {}".format(mean_squared_error(testY_inv, yhat_test_inv, squared=False)))
    print("Mean Absolute Percentage Error: {}".format(mean_absolute_percentage_error(testY_inv, yhat_test_inv)))
    
    plt.figure()
    plt.plot(yhat_test_inv, label='Predicted')
    plt.plot(testY_inv, label='True')
    plt.legend()
    plt.show()
    
    plt.figure()
    plt.plot(yhat_train_inv, label='Predicted')
    plt.plot(trainY_inv, label='True')
    plt.legend()
    plt.show()

In [19]:
run()

(158, 13)                                             
(140, 13, 1)                                          
(8, 13, 1)                                            
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

<ipython-input-4-a4ed99d749fe>:16: ExperimentalWarning: GCN_LSTM is experimental: Lack of unit tests and code refinement (see: https://github.com/stellargraph/stellargraph/issues/1132, https://github.com/stellargraph/stellargraph/issues/1526, https://github.com/stellargraph/stellargraph/issues/1564). It may be difficult to use and may have major changes at any time.
  model = GCN_LSTM(



0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Restoring model weights from the end of the best epoch: 1.
Epoch 00011: early stopping                           
Best validation loss of epoch:                        
0.12212962657213211                                   
(160, 13)                                                                        
(140, 13, 1)                                                                     
(8, 13, 1)                                                                       
  2%|▏         | 1/50 [00:15<12:45, 15.61s/trial, best loss: 0.12212962657213211]

...

0batch [00:00, ?batch/s]

Restoring model weights from the end of the best epoch: 5.                         
Epoch 00015: early stopping                                                        
Best validation loss of epoch:                                                     
0.0805649533867836                                                                 
100%|██████████| 50/50 [18:03<00:00, 21.66s/trial, best loss: 0.036123208701610565]
{'dropout': 0.30135100882461097, 'gcn_activation': 'tanh', 'gcn_layers': 1, 'gcn_units': 20, 'learning_rate': 0.056104038808002674, 'lstm_activation': 'tanh', 'lstm_layers': 1, 'lstm_units': 68, 'weeks': 1}
Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 13, 1)]           0         
                                                                 
 tf.expand_dims_11 (TFOpLamb  (None, 13, 1, 1)         0         
 da)                     

INFO:tensorflow:Assets written to: Model/GCN-LSTM\assets


INFO:tensorflow:Assets written to: Model/GCN-LSTM\assets


UnknownError: Failed to WriteFile: Model/GCN-LSTM\saved_model.pb.tmp5545be58598d4b17b016acb25958b6e8 : There is not enough space on the disk.
; Unknown error